In [4]:
import torch

In [6]:
def predict_gplinker(out, tags_id2name):
    entity, head, tail = out.split((2, len(tags_id2name), len(tags_id2name)), 1)
    idx = (entity > 0 ).nonzero()
    pre_entity = [[[] for i in range(2)] for _ in range(out.shape[0])]
    pre = [set() for _ in range(out.shape[0])]
    for batch, i, start, end in idx:
        pre_entity[batch][i].append((start, end))
    for batch, (object, subject) in enumerate(pre_entity):
        for s_o, e_o in object:
            for s_s, e_s in subject:
                idx_head = (head[batch, :, s_o, s_s] > 0).nonzero().squeeze(1)
                idx_tail = (tail[batch, :, e_o, e_s] > 0).nonzero().squeeze(1)
                # print(s_o, e_o, s_s, e_s, idx_head, idx_tail)
                for p in set(idx_head) & set(idx_tail):
                    pre[batch].add((s_o, e_o + 1, tags_id2name[p], s_s, e_s + 1))
    return pre

In [2]:
def get_label_gplinker(spo_batch):
    res = [set() for _ in len(spo_batch)]
    for i, item in enumerate(spo_batch):
        for s_o, e_o, p, s_s, e_s in item:
            res[i].add((s_o, e_o, p, s_s, e_s))
    return res

In [7]:
out = torch.randn(2, 34, 128, 128)
out[:, :2] = torch.triu(out[:, :2], diagonal=1)
tag_idx2name = {i:i for i in range(16)}
predict_gplinker(out, tag_idx2name)

KeyboardInterrupt: 

In [28]:
predict_gplinker(out, tag_idx2name)

[{(0, 2, 1, 0, 3),
  (0, 2, 1, 0, 4),
  (0, 2, 1, 0, 5),
  (0, 2, 3, 0, 3),
  (0, 2, 3, 0, 4),
  (0, 2, 3, 0, 5),
  (0, 2, 3, 1, 4),
  (0, 2, 3, 2, 4),
  (0, 2, 3, 3, 5),
  (0, 2, 5, 0, 3),
  (0, 2, 5, 0, 4),
  (0, 2, 5, 0, 5),
  (0, 2, 5, 1, 4),
  (0, 2, 7, 0, 5),
  (0, 2, 11, 0, 5),
  (0, 2, 11, 3, 5),
  (0, 2, 12, 1, 4),
  (0, 2, 12, 2, 4),
  (0, 2, 14, 0, 3),
  (0, 2, 14, 0, 5),
  (0, 2, 15, 0, 3),
  (0, 2, 15, 0, 4),
  (0, 2, 15, 1, 4),
  (0, 3, 1, 0, 3),
  (0, 3, 2, 1, 4),
  (0, 3, 2, 2, 4),
  (0, 3, 2, 3, 5),
  (0, 3, 3, 0, 4),
  (0, 3, 3, 0, 5),
  (0, 3, 3, 1, 4),
  (0, 3, 3, 2, 4),
  (0, 3, 3, 3, 5),
  (0, 3, 4, 0, 5),
  (0, 3, 4, 3, 5),
  (0, 3, 5, 0, 3),
  (0, 3, 5, 0, 4),
  (0, 3, 5, 1, 4),
  (0, 3, 7, 0, 4),
  (0, 3, 7, 0, 5),
  (0, 3, 7, 1, 4),
  (0, 3, 7, 2, 4),
  (0, 3, 9, 0, 3),
  (0, 3, 9, 0, 4),
  (0, 3, 9, 0, 5),
  (0, 3, 11, 0, 4),
  (0, 3, 11, 1, 4),
  (0, 3, 11, 2, 4),
  (0, 3, 14, 0, 3),
  (0, 3, 14, 0, 5),
  (0, 3, 15, 0, 4),
  (0, 3, 15, 1, 4),
  (0, 4, 1, 0, 

In [21]:
out[:, 2]

tensor([[[-0.0412, -1.1078, -0.4211, -1.1780, -1.1518],
         [ 0.5550, -0.7324, -0.5608, -0.0566, -0.5329],
         [ 1.0860,  1.1288, -1.1827,  0.8185,  0.1213],
         [-1.8950,  0.5243,  1.6568, -0.4791, -1.4212],
         [-0.0586, -0.3375,  0.9815,  1.5873, -0.9715]],

        [[-0.4448,  0.4877,  0.0229,  0.2115, -2.1286],
         [ 0.0790,  1.3136, -0.2235, -0.2311,  1.2407],
         [-1.3333, -0.6526,  1.0213, -1.2954,  1.1089],
         [-0.7337, -0.6817,  0.9775,  0.9582,  0.2944],
         [-0.4417, -0.7472, -0.1021,  1.0864,  0.4261]]])

In [2]:
a = torch.randn(2, 3, 4)
idx = (a>0).nonzero()
a, idx

(tensor([[[ 1.1987,  0.8780,  1.1522, -1.0931],
          [-0.9074, -0.3759, -0.1816, -0.0063],
          [-0.8904,  0.1558,  1.1008, -0.6462]],
 
         [[-0.2510,  0.9437,  0.7673,  0.4797],
          [-0.1367,  0.5009,  0.1517, -0.1847],
          [ 1.1735, -0.9008, -1.2535, -0.8222]]]),
 tensor([[0, 0, 0],
         [0, 0, 1],
         [0, 0, 2],
         [0, 2, 1],
         [0, 2, 2],
         [1, 0, 1],
         [1, 0, 2],
         [1, 0, 3],
         [1, 1, 1],
         [1, 1, 2],
         [1, 2, 0]]))